<a href="https://colab.research.google.com/github/BirukovAlex/neto_Python/blob/main/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BA_%D1%82%D0%B5%D0%BC%D0%B5_%C2%AB%D0%93%D0%B8%D0%B1%D1%80%D0%B8%D0%B4%D0%BD%D1%8B%D0%B5_%D1%80%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B%C2%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задача состоит в разработке гибридной рекомендательной системы по датасетам MovieLens. Результат оформим так: ID Пользователя, Понравившиеся фильмы, Рекомендуемые фильмы (% соответствия предпочтениям)

#Загрузка библиотек

In [1]:
!pip install git+https://github.com/daviddavo/lightfm

import numpy as np
import pandas as pd

from lightfm import LightFM

  Cloning https://github.com/daviddavo/lightfm to /tmp/pip-req-build-x0a1bif3
  Running command git clone --filter=blob:none --quiet https://github.com/daviddavo/lightfm /tmp/pip-req-build-x0a1bif3
  Resolved https://github.com/daviddavo/lightfm to commit f0eb500ead54ab65eb8e1b3890337a7223a35114
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp312-cp312-linux_x86_64.whl size=1099142 sha256=0767d62d6ae6a63749b815cddf54275fca4e5361a7f8e0e67771b12ff84f3a21
  Stored in directory: /tmp/pip-ephem-wheel-cache-h47db5ha/wheels/fd/89/93/70c1e5f378ee5043de89387ee3ef6852ff39e3b9eb44ecc1a3
Successfully built lightfm


In [3]:
import numpy as np
import pandas as pd

from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
from scipy.sparse import coo_matrix
import warnings
warnings.filterwarnings('ignore')

#Загрузка данных

In [5]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')
links = pd.read_csv('links.csv')

Загрузка данных...


#Предобработка данных

In [10]:
# Объединим фильмы с рейтингами
movie_ratings = pd.merge(ratings, movies, on='movieId')

# Для примера возьмем топ-500 самых оцениваемых фильмов
top_movies = movie_ratings['movieId'].value_counts().head(500).index
filtered_ratings = movie_ratings[movie_ratings['movieId'].isin(top_movies)]

# Отберем пользователей с достаточным количеством оценок
user_rating_counts = filtered_ratings['userId'].value_counts()
active_users = user_rating_counts[user_rating_counts >= 5].index
filtered_ratings = filtered_ratings[filtered_ratings['userId'].isin(active_users)]

Предобработка данных...


#Подготовка фичей

In [11]:
# Жанры как фичи для элементов
movies['genres'] = movies['genres'].str.split('|')
all_genres = set()
for genres in movies['genres'].dropna():
    if isinstance(genres, list):
        all_genres.update(genres)
all_genres = list(all_genres)

# Создаем датасет LightFM
dataset = Dataset()
dataset.fit(
    users=filtered_ratings['userId'].unique(),
    items=filtered_ratings['movieId'].unique(),
    item_features=all_genres
)

# Создаем матрицу взаимодействий
interactions, weights = dataset.build_interactions(
    [(row['userId'], row['movieId'], row['rating'])
     for _, row in filtered_ratings.iterrows()]
)

# Создаем фичи только для фильмов, которые есть в нашем датасете
movie_features = []
unique_movies_in_dataset = filtered_ratings['movieId'].unique()

for movie_id in unique_movies_in_dataset:
    movie_info = movies[movies['movieId'] == movie_id]
    if len(movie_info) > 0:
        movie_genres = movie_info['genres'].values[0]
        if isinstance(movie_genres, list):
            movie_features.append((movie_id, movie_genres))
        else:
            movie_features.append((movie_id, []))
    else:
        movie_features.append((movie_id, []))

# Строим фичи элементов
item_features = dataset.build_item_features(movie_features)


Подготовка фичей...


#Проверяем размерности (во избежание exeption)

In [12]:
print(f"Количество фильмов в interactions: {interactions.shape[1]}")
print(f"Количество фильмов в item_features: {item_features.shape[0]}")

Количество фильмов в interactions: 500
Количество фильмов в item_features: 500


#Обучаем модель

In [13]:
model = LightFM(loss='warp', no_components=20, learning_rate=0.05)
model.fit(interactions, item_features=item_features, epochs=20, num_threads=2, verbose=True)

Обучение модели...


Epoch: 100%|██████████| 20/20 [00:14<00:00,  1.37it/s]


#Генерируем рекоммендации

In [14]:
# Создаем mapping для быстрого поиска
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

# Для демонстрации возьмем первых 10 пользователей
user_ids = list(filtered_ratings['userId'].unique()[:10])
recommendations = {}

for user_id in user_ids:
    try:
        # Фильмы, которые пользователь уже оценил высоко (>3.5)
        user_ratings = filtered_ratings[filtered_ratings['userId'] == user_id]
        liked_movies = user_ratings[
            user_ratings['rating'] > 3.5
        ]['title'].tolist()[:5]  # Берем первые 5 понравившихся

        # Если нет высоких оценок, берем просто просмотренные
        if not liked_movies:
            liked_movies = user_ratings['title'].tolist()[:3]

        # Предсказание рейтингов для всех фильмов
        all_items = list(item_id_map.keys())
        user_index = user_id_map[user_id]

        scores = model.predict(user_index, np.array(list(item_id_map.values())),
                             item_features=item_features)

        # Исключаем уже просмотренные фильмы
        viewed_movies = user_ratings['movieId'].values
        viewed_indices = [item_id_map[movie_id] for movie_id in viewed_movies
                         if movie_id in item_id_map]

        # Создаем маску для непросмотренных фильмов
        mask = np.ones(len(all_items), dtype=bool)
        mask[viewed_indices] = False

        available_scores = scores[mask]
        available_items = np.array(all_items)[mask]

        # Топ-5 рекомендаций
        if len(available_scores) > 0:
            top_indices = np.argsort(-available_scores)[:5]
            recommended_movie_ids = available_items[top_indices]
            recommendation_scores = available_scores[top_indices]

            # Преобразуем scores в проценты (нормализуем)
            max_score = np.max(scores) if len(scores) > 0 else 1
            min_score = np.min(scores) if len(scores) > 0 else 0
            if max_score > min_score:
                accuracy_percentages = ((recommendation_scores - min_score) /
                                      (max_score - min_score) * 100).astype(int)
            else:
                accuracy_percentages = np.array([80, 75, 70, 65, 60])  # fallback

            # Получаем названия рекомендованных фильмов
            recommended_movies = []
            for movie_id, accuracy in zip(recommended_movie_ids, accuracy_percentages):
                movie_title = movies[movies['movieId'] == movie_id]['title'].values
                if len(movie_title) > 0:
                    recommended_movies.append((movie_title[0], accuracy))
                else:
                    recommended_movies.append((f"Фильм ID: {movie_id}", accuracy))

            recommendations[user_id] = {
                'liked_movies': liked_movies,
                'recommended_movies': recommended_movies
            }
        else:
            recommendations[user_id] = {
                'liked_movies': liked_movies,
                'recommended_movies': [("Недостаточно данных для рекомендаций", 0)]
            }

    except Exception as e:
        print(f"Ошибка для пользователя {user_id}: {e}")
        recommendations[user_id] = {
            'liked_movies': [],
            'recommended_movies': [("Ошибка генерации рекомендаций", 0)]
        }

Генерация рекомендаций...


#Формируем итоговую таблицу

In [18]:
# 7. Итоговые данные + лаконичный дизайн
print("\n" + "╔" + "═" * 98 + "╗")
print("║" + "🎬 РЕКОМЕНДАТЕЛЬНАЯ СИСТЕМА: ПЕРСОНАЛЬНЫЕ РЕКОМЕНДАЦИИ".center(98) + "║")
print("╚" + "═" * 98 + "╝")

for user_id, data in recommendations.items():
    print(f"\n┌{'─' * 96}┐")
    print(f"│ 👤 Пользователь #{user_id}".ljust(97) + "│")
    print(f"├{'─' * 96}┤")

    # Секция понравившихся фильмов
    print(f"│ ❤️  Понравившиеся фильмы:".ljust(97) + "│")
    if data['liked_movies']:
        for movie in data['liked_movies']:
            print(f"│    🎭 {movie}".ljust(97) + "│")
    else:
        print(f"│    📊 Недостаточно данных о предпочтениях".ljust(97) + "│")

    print(f"├{'─' * 96}┤")

    # Секция рекомендаций
    print(f"│ 🎯 Рекомендуемые фильмы:".ljust(97) + "│")
    for movie, accuracy in data['recommended_movies']:
        if accuracy >= 80:
            indicator = "🟢 Высокое соответствие"
        elif accuracy >= 60:
            indicator = "🟡 Среднее соответствие"
        else:
            indicator = "🔴 Базовые рекомендации"

        print(f"│    ⭐ {movie}".ljust(97) + "│")
        print(f"│       📊 Точность: {accuracy}% ({indicator})".ljust(97) + "│")
        print(f"│".ljust(97) + "│")

    print(f"└{'─' * 96}┘")

print("\n" + "📊 Метрика точности показывает, насколько рекомендация соответствует вашим вкусам")


╔══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                      🎬 РЕКОМЕНДАТЕЛЬНАЯ СИСТЕМА: ПЕРСОНАЛЬНЫЕ РЕКОМЕНДАЦИИ                       ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════╝

┌────────────────────────────────────────────────────────────────────────────────────────────────┐
│ 👤 Пользователь #1                                                                              │
├────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ❤️  Понравившиеся фильмы:                                                                      │
│    🎭 Toy Story (1995)                                                                          │
│    🎭 Grumpier Old Men (1995)                                                                   │
│    🎭 Heat (1995)                                                                               │
│ 

#Всякая техническая информация

In [16]:
# 8. Дополнительная информация о качестве модели
print("\n" + "="*80)
print("ИНФОРМАЦИЯ О КАЧЕСТВЕ МОДЕЛИ")
print("="*80)

try:
    # Оценка precision@k
    train_precision = precision_at_k(model, interactions,
                                   item_features=item_features, k=5).mean()
    print(f"Precision@5 на обучающих данных: {train_precision:.2%}")
except:
    print("Не удалось вычислить precision@k")

print("\nМетрика 'Соответствие предпочтениям' показывает,")
print("насколько рекомендация соответствует вашим вкусам на основе:")
print("- Ваших предыдущих оценок")
print("- Схожести с понравившимися вам фильмами")
print("- Популярности среди пользователей с похожими предпочтениями")
print("- Жанровых предпочтений")


ИНФОРМАЦИЯ О КАЧЕСТВЕ МОДЕЛИ
Precision@5 на обучающих данных: 78.05%

Метрика 'Соответствие предпочтениям' показывает,
насколько рекомендация соответствует вашим вкусам на основе:
- Ваших предыдущих оценок
- Схожести с понравившимися вам фильмами
- Популярности среди пользователей с похожими предпочтениями
- Жанровых предпочтений


In [17]:
# 9. Дополнительная статистика
print("\n" + "="*80)
print("СТАТИСТИКА ДАННЫХ")
print("="*80)
print(f"Всего пользователей: {len(filtered_ratings['userId'].unique())}")
print(f"Всего фильмов: {len(filtered_ratings['movieId'].unique())}")
print(f"Всего оценок: {len(filtered_ratings)}")
print(f"Уникальных жанров: {len(all_genres)}")


СТАТИСТИКА ДАННЫХ
Всего пользователей: 596
Всего фильмов: 500
Всего оценок: 43701
Уникальных жанров: 0
